# **Flocculator Redesign Workbook** 

In [3]:
!pip install aguaclara
import aguaclara.core.head_loss as minorloss
import aguaclara.design.human_access as ha
import aguaclara.core.physchem as pc
import aguaclara.core.pipes as pipes
from aguaclara.core.units import unit_registry as u
import numpy as np
import pandas as pd

In [4]:
Pi = 3
Q = 720 * u.cubic_feet/u.s
K = 4
W = 4 * u.m
viscosity = pc.viscosity_kinematic_water(25*u.degC)
Gt = 37000
theta = 7.5 *u.min
G = Gt/theta
H = ((Pi**3*Q**3*K)/(W**3*2*viscosity*G**2))**(1/4)
print(H.to(u.m))

32.99 meter


Flocculation is widely used in water treatment because of the use of sedimentation to remove particles from the water. Flocculation aims to increase the diameter of the particles through applying coagulant during rapid mix. In order to facilitate particle collisions, we must design a flocculator that takes into account collision potential and energy dissipation rate.
In our analysis, we will take into account the number of treatment trains, variation of flow rates, turbidity, and downstream H of the flocculation tank. In our design, we will use a horizontal hydraulic flocculator. We will use a residence time of 7.5 minutes, minimum width of 4 meters, and a flow rate of 720 cubic feet per second.
Outputs from our analysis will include the plan view area of the tank, the length of the channels, width of the channel, width and spacing of the baffles.

New Flocculator Design

In [7]:
##HydraluicFloculator Code
class HydraluicFloculator:
    """Calculates physical dimensions of a HydraluicFloculator
    ----------------------------
    - BAFFLE_K (K or K_{baffle}): float
        - The minor loss coefficient of the flocculator baffles.
    - CHANNEL_N_MIN (N_{channel}: int
        - The minimum number of flocculator channels.
    - WS_RATIO_MIN (\Pi_{WS}): float
        - The minimum ratio between expansion height and baffle spacing
    - RATIO_MAX_WS (\Pi_{WS}): float
        - The maximum ratio between expansion height and baffle spacing
    - SDR (sdr): float
        - The standard dimension ratio.
    """

    # Increased both to provide a safety margin on flocculator head loss and
    # to simultaneously scale back on the actual collision potential we are
    # trying to achieve.
    # Originally calculated to be 2.3 from the equations:


    BAFFLE_K = 4
    CHANNEL_N_MIN = 1
    RATIO_MAX_WS = 3
    SDR = 41.0

    def __init__(
            self,
            Q=1 * u.L/u.s,
            temp=25 * u.degC,
            Gt=37000,
            HL = 40 * u.cm,
            downstream_H = 2 * u.m,
            channel_H = 4 * u.m,
            n_treatment_trains = 1,
            ):

        self.Q = Q
        self.temp = temp
        self.Gt = Gt
        self.HL = HL
        self.downstream_H = downstream_H
        self.channel_H = channel_H
        self.n_treatment_trains = n_treatment_trains


    @property
    def vel_grad_avg(self):
        #NOTE: this is borrowed from AguaClara source code
        """Calculate the average velocity gradient (G-bar) of water flowing
        through the flocculator.
        :returns: Average velocity gradient (G-bar)
        :rtype: float * 1 / second
        """
        return ((u.standard_gravity * self.HL) /
               (pc.viscosity_kinematic_water(self.temp) * self.Gt)).to(u.s ** -1)

    @property
    def retention_time(self):
        #NOTE: this is borrowed from AguaClara source code
        """Calculates the hydraulic retention time neglecting the volume created by head loss in the flocculator.
        :returns: Hydraulic retention time (:math:`\theta`)
        :rtype: float * second
        """
        return (self.Gt / self.vel_grad_avg).to(u.s)

    @property
    def total_vol(self):
        #NOTE: this is borrowed from AguaClara source code
        """Calculate the target volume (not counting the volume added by head loss) of the flocculator.
        :returns: Target volume
        :rtype: float * meter ** 3
        """
        return (self.Q * self.retention_time).to(u.m ** 3)

    @property
    def treatment_train_vol(self):
        #NOTE: this is borrowed from AguaClara source code
        """Calculate the target volume (not counting the volume added by head loss) of the flocculator.
        :returns: Target volume
        :rtype: float * meter ** 3
        """
        return (self.total_vol/self.n_treatment_trains).to(u.m ** 3)

    @property
    def baffle_W(self):
       """This function calculates the baffle width by multiplying by the maximum width to spacing ratio"""
       return (((self.RATIO_MAX_WS**3*self.Q**3*self.BAFFLE_K)/
                (self.channel_H**3*2*pc.viscosity_kinematic_water(self.temp)*self.vel_grad_avg**2))**(1/4)).to(u.m)

    @property
    def baffle_S(self):
      #NOTE: This is borrowed from AguaClara source code
        """Return the spacing between baffles.
        :returns: Spacing between baffles
        :rtype: int
        """
        return (self.baffle_W/3).to(u.m)

    @property
    def channel_W(self):
       """This function calculates the channel width by adding baffle width and baffle spacing"""
       return (self.baffle_W+self.baffle_S).to(u.m)


    @property
    def channel_L(self):
        """This function determines channel length based the largest square that fits inside of the selected tank. So, channel length is the the length of the square"""
        return (self.treatment_train_vol/(self.channel_W*self.channel_H)).to(u.m)
        
    @property
    def side_view_area(self):
        """This function calculates the necessary active cross sectional area of the flocculator"""
        return (self.treatment_train_vol/self.channel_W).to(u.m**2)

    @property
    def plan_view_area(self):
        """This function calculates the plan view area of the flocculator"""
        return (self.treatment_train_vol/self.channel_H).to(u.m**2)

    @property
    def baffle_n(self):
        """This function calclulates the number of the baffles in the flocculator."""
        return (np.ceil((self.channel_L/self.baffle_S))).to(u.dimensionless)

    @property
    def vel_scour(self):
        """This function calclulates the number of the baffles in the flocculator."""
        return (self.Q/(self.channel_H*self.baffle_S)).to(u.m/u.s)

    @property
    def design(self):
        """Returns the designed values.
        :returns: list of designed values (G, t, channel_W, obstacle_n)
        :rtype: int
        """
        floc_dict = {'total_vol': self.total_vol,
                     'treatment_train_vol': self.treatment_train_vol,
                     'side_view_area': self.side_view_area,
                     'plan_view_area': self.plan_view_area,
                     'channel_L': self.channel_L,
                     'channel_W': self.channel_W,
                     'channel_H': self.channel_H,
                     'baffle_S': self.baffle_S,
                     'baffle_W': self.baffle_W,
                     'baffle_n': self.baffle_n,
                     'G': self.vel_grad_avg,
                     't': self.retention_time,
                     'vel_scour': self.vel_scour}
        return floc_dict


##make a data frame to show results
q  = 720 * u.cubic_feet/u.s #change this flow rate in order to display design specs for the HydraluicFloculator handling that flow rate
tester=HydraluicFloculator(Q=q)
#print(type(HydraluicFloculator(n_treatment_trains=1, Q=q).total_vol))

row_names = ['Total Volume', 
                'Volume of Treatment Train', 
                'Side View Area', 
                'Plan View Area', 
                'Channel Length', 
                'Channel Width', 
                'Channel Height', 
                'Baffle Spacing', 
                'Baffle Width', 
                'Number of Baffles',
                'Scour Velocity']
  
df=pd.DataFrame()
for i in range(1, 9):
  data = [HydraluicFloculator(n_treatment_trains=i, Q=q).total_vol,
          HydraluicFloculator(n_treatment_trains=i, Q=q).treatment_train_vol,
          HydraluicFloculator(n_treatment_trains=i, Q=q).side_view_area,
          HydraluicFloculator(n_treatment_trains=i, Q=q).plan_view_area,
          HydraluicFloculator(n_treatment_trains=i, Q=q).channel_L,
          HydraluicFloculator(n_treatment_trains=i, Q=q).channel_W,
          HydraluicFloculator(n_treatment_trains=i, Q=q).channel_H,
          HydraluicFloculator(n_treatment_trains=i, Q=q).baffle_S,
          HydraluicFloculator(n_treatment_trains=i, Q=q).baffle_W,
          HydraluicFloculator(n_treatment_trains=i, Q=q).baffle_n,
          HydraluicFloculator(n_treatment_trains=i, Q=q).vel_scour]
  df[str(i)+' Treatment Trains'] = pd.Series(data,index=row_names)
print(df)


class Floc_7_5_min_res(HydraluicFloculator):
  """This class models a HydraluicFloculator with 1 minute residence time"""
  @property
  def retention_time(self):
    return 7.5*u.min
  @property
  def vol(self):
    return ((self.Q*self.retention_time).to(u.L))


                             1 Treatment Trains  ...    8 Treatment Trains
Total Volume                    6354 meter ** 3  ...       6354 meter ** 3
Volume of Treatment Train       6354 meter ** 3  ...      794.3 meter ** 3
Side View Area                 173.6 meter ** 2  ...       21.7 meter ** 2
Plan View Area                  1589 meter ** 2  ...      198.6 meter ** 2
Channel Length                       43.4 meter  ...           5.425 meter
Channel Width                       36.61 meter  ...           36.61 meter
Channel Height                          4 meter  ...               4 meter
Baffle Spacing                      9.152 meter  ...           9.152 meter
Baffle Width                        27.45 meter  ...           27.45 meter
Number of Baffles               5 dimensionless  ...       1 dimensionless
Scour Velocity             0.557 meter / second  ...  0.557 meter / second

[11 rows x 8 columns]
